In [ ]:
import numpy as np
from PySDM import Formulae
from PySDM.physics import si
from PySDM.initialisation.spectra import Lognormal
from PySDM.products import (
    ParcelDisplacement,
    AmbientTemperature,
    AmbientDryAirDensity,
    AmbientRelativeHumidity,
    ParticleSizeSpectrumPerVolume,
    ParticleVolumeVersusRadiusLogarithmSpectrum,
)

from PySDM_examples.Pyrcel import Settings, Simulation
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
exporters_path = os.path.join(notebook_dir, "../../../PySDM/exporters/")
print(exporters_path)
sys.path.append(exporters_path)

from parcel_vtk_exporter import VTKExporterPyrcel

c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\../../../PySDM/exporters/


In [8]:
settings = Settings(
    dz=10 * si.m,
    n_sd_per_mode=(1, 1),
    aerosol_modes_by_kappa={
        0.54: Lognormal(norm_factor=850 / si.cm**3, m_mode=15 * si.nm, s_geom=1.6),
        1.2: Lognormal(norm_factor=10 / si.cm**3, m_mode=850 * si.nm, s_geom=1.2),
    },
    vertical_velocity=1.0 * si.m / si.s,
    initial_pressure=775 * si.mbar,
    initial_temperature=274 * si.K,
    initial_relative_humidity=0.98,
    displacement=1000 * si.m,
    formulae=Formulae(constants={"MAC": 0.3}),
)

dry_radius_bin_edges = np.logspace(
    np.log10(1e-3 * si.um), np.log10(5e0 * si.um), 33, endpoint=False
)

simulation = Simulation(
    settings,
    products=(
        ParcelDisplacement(name="z"),
        AmbientRelativeHumidity(name="S_max_percent", unit="%", var="RH"),
        AmbientTemperature(name="T"),
        ParticleSizeSpectrumPerVolume(
            name="dry:dN/dR", radius_bins_edges=dry_radius_bin_edges, dry=True
        ),
        ParticleVolumeVersusRadiusLogarithmSpectrum(
            name="dry:dV/dlnR", radius_bins_edges=dry_radius_bin_edges, dry=True
        ),
        AmbientDryAirDensity(),
    ),
)

In [9]:
output = simulation.run()

In [10]:
e = VTKExporterPyrcel(n_sd=simulation.particulator.n_sd)
for step in settings.output_steps:
    e.export_products(output, step, simulation)
    e.export_attributes(output, step, simulation)
e.write_pvd()

c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000000.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000001.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000002.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000003.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000004.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000005.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000006.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000007.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000008.vtu
c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\output\sd_attributes_num0000000009.vtu
c:\Users\s